This code saves two files:
1)     faves.csv
    table containing over 1000 of most popular books, bands, movies and foods and counts for each user.
    Should I come back and scale these by idf?
    http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/
2)     faves_svd.csv
    table containing 3 principle components of faves following truncated SVD analysis

In [77]:
import pandas as pd
import re
import scipy as sp
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [2]:
okc = pd.read_csv('../Assets/A/train.csv')

In [3]:
# Remove markdown

def essay_to_words( raw_essay ):

    # 1. Remove HTML
    text = BeautifulSoup(raw_essay, 'lxml').get_text() 
    #
    # 2. Remove Newline characters       
    text2 = text.replace('\n', ' ')
    return text2

Categories are:
books, movies, tv shows, music, food

In [4]:
# Save counter to include top books, movies, tv, music and food
faves_count = Counter()

# Save counters for all words that come before a colon
# There may be common variations on the above categories that have been overlooked
# Saving categories will help refine regex for search below
cats = Counter()

In [5]:
# Takes regex search object and saves items from it to chosen list

def split_n_save(search_obj, cnt):

    # remove first word (category e.g. 'books:') last word (next category e.g. 'movies:')
    # split list by commas and remove white space
    for item in search_obj.split(' ',1)[1].rsplit(' ',1)[0].split(','):
        clean_item = item.strip()
        cnt[clean_item] += 1 


### Search for movies, Tv Shows, Music, and food

In [6]:
SA = okc.essay4[0]

In [7]:
SA

"book:<br />\n-gone with the wind - margaret mitchell<br />\n-norwegian wood - haruki murakami<br />\n-nine stories - j. d. salinger<br />\n-the wall - jean-paul sartre<br />\n-villa incognito - tom robbins<br />\n<br />\nmovie:<br />\n-the shop around the corner<br />\n-the matrix<br />\n-the little mermaid<br />\n-spirited away<br />\n-the adventures of baron munchausen<br />\n<br />\nshow:<br />\n-lost<br />\n-once upon a time<br />\n-battlestar galactica<br />\n-firefly<br />\n-coffee prince<br />\n<br />\nmusic:<br />\n-muse<br />\n-daft punk<br />\n-kimbra<br />\n-ludwig van beethoven<br />\n-george gershwin<br />\n<br />\nfood:<br />\n-street tacos<br />\n-french bistro<br />\n-japanese noodle truck<br />\n-farmer's market samplings<br />\n-red meat and dark chocolate"

In [8]:
def find_lists(essay):
    
    # Skip empty essays
    # Remove markdown from the rest
    # Do any of these have capitol letters?

    try:
        SA = essay_to_words(essay)
    except:
        return ''
    

    # Save lists for each category
    # All strings in Data Set A are lower case
    # Most in Data Set B contain lower and upper case
    
    try:
        fave_list = re.search('''[Bb]ooks?:(.*?)(:|$)|Rreading:(.*?)(:|$)|[Aa]uthors:(.*?)(:|$)
        |[Mm]ovies?:(.*?)(:|$)|[fF]ilms?:(.*?)(:|$)([Tt][Vv]|[Ss]hows):(.*?)(:|$)([Mm]usic|[Bb]ands):
        (.*?)(:|$)[Ff]oods?:(.*?)(:|$)''', SA).group(0)
        split_n_save(fave_list, faves_count)
    except:
        pass
   
    # Save other categories for later viewing
    try:
        for cat in re.findall('(\S*):', SA):
            cats[cat] += 1
    except:
        pass
    return essay

In [9]:
# Why does this keep printing 'None' for every single essay?

# Save urls for later analysis?
okc['essay4'] = okc.essay4.apply(find_lists)

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://thefeministwire.com/2012/02/of-ambivalence-the-help-obama-and-the-ultimate-slave-re-mix/#.t047sgk-meo.facebook" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You

In [10]:
faves_count

Counter({u'': 552,
         u'written by true and faithful christian authors that writes only for the glory of god. i also love books about nature and books about enjoying': 1,
         u'daniel quinn': 1,
         u'consider phlebas': 3,
         u'short stories by karen russell and jhumpa lahiri.': 1,
         u'finding nemo and peter pan i like watch ballet': 1,
         u'random advertising textbooks (lame': 1,
         u'limp wrist': 1,
         u'the original star wars trilogy is hard to beat.': 1,
         u'i dont do it as much as i used to sadly. i am really big into fantasy but do enjoy a more serious read as well. i am also a comic book nerd': 1,
         u'a lot more.': 1,
         u'on the road.': 1,
         u'peril of magnificent love by emma magenta': 1,
         u'black hole': 6,
         u'a sample of my': 1,
         u'the perk of being a wallflower': 1,
         u'torpedo juice.': 1,
         u'any erick jerome dickey': 1,
         u'when god was a woman': 1,
      

In [11]:
# Select all items in counter that occurred more than 10 times
# Remove '' from each list

def most_pop(counter):
    top = filter(lambda x: x[1] >= 10, counter.most_common())
    top_copy = top
    for item in top:
        if item[0] == '':
            top_copy.remove(item)
    return top_copy

In [12]:
top_faves = most_pop(faves_count)

In [13]:
len(top_faves)

1067

In [14]:
# Extract feature names from top_faves
features = [fave[0] for fave in top_faves]

In [17]:
# Build new df with dummies for cols for all items liked by more than 10 people

faves = pd.DataFrame(index=okc.index, columns=features)

In [18]:
# Iterate through users and count features in essay 4

# Can I write straight to scipy sparse?
# I should use sklearn tf-idf but build custom tokenizer to look for just specific words.

# This will take several hours.  Run overnight

for user in faves.index:
    for col in faves.columns:
        # search essay4 for each feature    
        faves.ix[user, col] = okc.ix[user,'essay4'].count(col)

In [20]:
faves.to_csv('../Assets/A/faves.csv')

In [67]:
faves.shape

(53951, 1067)

In [81]:
type(faves)

pandas.core.frame.DataFrame

In [86]:
faves = sp.sparse.csc_matrix(faves.values)

In [87]:
# Perform truncated SVD
# I believe it is preferable not to attempt to scale sparse data prior to SVD?????
# http://scikit-learn.org/stable/modules/preprocessing.html#scaling-sparse-data

svd = TruncatedSVD(n_components=3, random_state=42)
faves_svd = svd.fit_transform(faves)

In [88]:
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

[  9.99986701e-01   1.89061191e-07   1.93562802e-10]
0.999986890508


SHOULD I SCALE AFTER SVD????
This guy does it:
http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

Proceeding forward with data NOT SCALED OR NORMALIZED!

In [89]:
faves_svd_df = pd.DataFrame(faves_svd)
faves_svd_df.to_csv('../Assets/A/faves_SVD.csv')

Over 98% of variance is explained by first component!  Check that out!  Reduced my number of features from 100(recommended for text classfication) to 3.  Still explaining over 99% of the variance.  Graph these? 

In [35]:
# Unpack what's in here!  Explore covariance with features from toy_faves
# Remember this is not predictive of anything just eda
svd.components_[0]


array([  6.73270415e-04,   6.21634821e-04,   3.68154055e-04, ...,
         1.23108070e-05,   1.75726869e-05,   2.44622564e-05])

### Try clustering bands using kmeans
http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

In [48]:
type(faves_svd)

pandas.core.frame.DataFrame

In [40]:
# Build multinomialNB classifier for sex to test if this works
# Can only model features that do not contain null values here.  
# Must come up with protocol for eliminating nulls to model most features.

# CANNOT USE MultinomialNB post-SVD.  NB requires positive integer features.
# Can I use GaussianNB?  Doesn't sound like a good idea since features are multinomially distributed, not normally

from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score

### Build model with GaussianNB
and a high dose of skepticism

In [43]:
nb = GaussianNB()
y = np.ravel(okc.sex)

In [47]:
y.shape

(53951,)

In [51]:
faves_svd.shape

(53951, 3)

In [64]:
cv = cross_val_score(nb, faves_svd, y=y, scoring='accuracy',  n_jobs=3)

In [66]:
cv

array([ 0.44086739,  0.45921148,  0.44592115])

The GaussianNB model is WAY WORSE THAN RANDOM!

### Try Textacy's text classification Model

In [ ]:
y[train_index]

In [ ]:
faves_svd[train_index]

### Use spaCy to calculate "reading level" of essays and compare that to favorite books/movies, etc.
Compare to race, class and income and discuss potential algorithmic biases